In [1]:
import numpy as np
from numba import jit
from scipy.ndimage import label
from tifffile import imread, imsave
from glob import glob
from scipy import ndimage
from skimage.measure import label

In [2]:
@jit
def pixel_sharing_bipartite(lab1, lab2):
    assert lab1.shape == lab2.shape
    psg = np.zeros((lab1.max()+1, lab2.max()+1), dtype=np.int)
    for i in range(lab1.size):
        psg[lab1.flat[i], lab2.flat[i]] += 1
    return psg

In [3]:
def matching_iou(psg, fraction=0.5):
    iou = intersection_over_union(psg)
    matching = iou > fraction
    matching[:,0] = False
    matching[0,:] = False
    return matching

In [4]:
def matching_max(psg):
    """
    matching based on mutual first preference
    """
    rowmax = np.argmax(psg, axis=0)
    colmax = np.argmax(psg, axis=1)
    starting_index = np.arange(psg.shape[1])
    equal_matches = colmax[rowmax[starting_index]]==starting_index
    rm, cm = rowmax[equal_matches], colmax[rowmax[equal_matches]]
    matching = np.zeros_like(psg)
    matching[rm, cm] = 1
    return matching

In [5]:
def intersection_over_union(psg):
    rsum = np.sum(psg, 0, keepdims=True)
    csum = np.sum(psg, 1, keepdims=True)
    return psg / (rsum + csum - psg)

In [6]:
def matching_overlap(psg, fractions=(0.5,0.5)):
    """
    create a matching given pixel_sharing_bipartite of two label images based on mutually overlapping regions of sufficient size.
    NOTE: a true matching is only gauranteed for fractions > 0.5. Otherwise some cells might have deg=2 or more.
    NOTE: doesnt break when the fraction of pixels matching is a ratio only slightly great than 0.5? (but rounds to 0.5 with float64?)
    """
    afrac, bfrac = fractions
    set0_object_sizes = np.sum(psg, axis=1, keepdims=True)
    m0  = np.where(set0_object_sizes==0,0,psg / set0_object_sizes)
    set1_object_sizes = np.sum(psg, axis=0, keepdims=True)
    m1 = np.where(set1_object_sizes==0,0,psg / set1_object_sizes)
    m0 = m0 > afrac
    m1 = m1 > bfrac
    matching = m0 * m1
    matching = matching.astype('bool')
    return matching

In [137]:
def precision(lab_gt, lab, iou=0.5, partial_dataset=False):
    '''
    precision = TP / (TP + FP + FN) i.e. "intersection over union" for a graph matching
    '''
    
    psg = pixel_sharing_bipartite(lab_gt, lab)
    matching = matching_iou(psg, fraction=iou)
    assert matching.sum(0).max() < 2
    assert matching.sum(1).max() < 2
    n_gt  = len(set(np.unique(lab_gt)) - {0})
    n_hyp = len(set(np.unique(lab)) - {0})
    n_matched = matching.sum()
#     print("TP:", n_matched, "FP:", n_hyp-n_matched, "FN:", n_gt-n_matched)
    if partial_dataset:
        return n_matched , (n_gt + n_hyp - n_matched)
    else:
        return n_matched / (n_gt + n_hyp - n_matched)

In [124]:
## full scores
def seg(lab_gt, lab, partial_dataset=False):
    """
    calculate seg from pixel_sharing_bipartite
    seg is the average conditional-iou across ground truth cells
    conditional-iou gives zero if not in matching
    ----
    calculate conditional intersection over union (CIoU) from matching & pixel_sharing_bipartite
    for a fraction > 0.5 matching. Any CIoU between matching pairs will be > 1/3. But there may be some
    IoU as low as 1/2 that don't match, and thus have CIoU = 0.
    """
    psg = pixel_sharing_bipartite(lab_gt, lab)
    iou = intersection_over_union(psg)
    matching = matching_overlap(psg, fractions=(0.5, 0))
    matching[0,:] = False
    matching[:,0] = False
    n_gt = len(set(np.unique(lab_gt)) - {0})
    n_matched = iou[matching].sum()
    if partial_dataset:
        return n_matched , n_gt
    else:
        return n_matched / n_gt, (2* (n_matched / n_gt)/(1+(n_matched / n_gt)))

In [125]:
gt_segs_path = glob("/Users/prakash/Desktop/MetaSeg_Data_Version1/CTC_GOWT1/SEG/*.tif")
label_segs_path = glob("/Users/prakash/Desktop/MetaSeg_Data_Version1/CTC_GOWT1/BIC/*.tif")

seg_scores_per_time = []
dice_scores_per_time = []
for i in range(len(gt_segs_path)):
    seg_score, dice_score = seg(imread(gt_segs_path[i]), imread(label_segs_path[i]).astype(np.uint16))
#     print(seg_score, dice_score)
    seg_scores_per_time.append(seg_score)
    dice_scores_per_time.append(dice_score)

print("SEG score :", np.mean(np.array(seg_scores_per_time) ))
print("DICE score :", np.mean(np.array(dice_scores_per_time) ) )

/Users/prakash/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in true_divide
  # This is added back by InteractiveShellApp.init_path()


SEG score : 0.79259697644
DICE score : 0.882969141742


In [157]:
average_precision_score_list = []
thresholds = np.arange(0.5,0.95,0.05)

for thresh in thresholds:
    gt_segs_path = glob("/Users/prakash/Desktop/MetaSeg_Data_Version1/CTC_GOWT1/SEG/*.tif")[:50]
    label_segs_path = glob("/Users/prakash/Desktop/MetaSeg_Data_Version1/CTC_GOWT1/runs/run3/50annotations/*.tif")[:50]
    sum_seg = 0
    scores_per_time = []
    for i in range(len(gt_segs_path)):
        score = precision(imread(label_segs_path[i]).astype(np.uint16), imread(gt_segs_path[i]), thresh)
        scores_per_time.append(score)
        if(0<=score<=1):
            sum_seg = sum_seg + score
            
    average_precision_score = sum_seg/len(gt_segs_path)
    average_precision_score_list.append(average_precision_score)
    
#     print("AP score for threshold "+str(thresh)+" :", average_precision_score )
    
print("Mean AP score is: ", np.mean(np.array(average_precision_score_list)))

Mean AP score is:  0.704688041852


In [158]:
average_precision_score_list = []
thresholds = np.arange(0.5,0.95,0.05)
for thresh in thresholds:
    
    gt_segs_path = glob("/Users/prakash/Desktop/MetaSeg_Data_Version1/CTC_GOWT1/SEG/*.tif")[:50]
    label_segs_path = glob("/Users/prakash/Desktop/MetaSeg_Data_Version1/CTC_GOWT1/BIC/*.tif")[:50]
    sum_seg = 0
    scores_per_time = []
    for i in range(len(gt_segs_path)):
        score = precision(imread(label_segs_path[i]).astype(np.uint16), imread(gt_segs_path[i]), thresh)
        scores_per_time.append(score)
        if(0<=score<=1):
            sum_seg = sum_seg + score
            
    average_precision_score = sum_seg/len(gt_segs_path)
    average_precision_score_list.append(average_precision_score)
#     print("AP score for threshold "+str(thresh)+" :", average_precision_score )

print("Mean AP score is: ", np.mean(np.array(average_precision_score_list)))

Mean AP score is:  0.692107628024
